In [ ]:
# simple 0D Energy Balance Model

import numpy as np
from matplotlib import pyplot as plt
from mod1 import qnet_toa,ebm,ebm_bifurcation_diagram
from mod1 import ebm_fixed_points,feedback_factors

In [ ]:
# (a) sensitivity to initial conditions: forward integration

ho=10.0 # mixed layer depth
epsilon=0.61 # planetary emissivity tuned to match the observed
#              surface temperature 288 K with present-day insolation
#              S0=1365 W/m2). epsilon=(Te/Ts_observed)**4,
#              with Te=255 K the emission temperature
S0=1365.0 # solar constant (W/m2)
nci=51 # number of initial conditions

# choose albedo type here
alpha_type='nonlinear' # uniform, linear or nonlinear
tsi=np.linspace(230,300,nci)
nt=1000;temp=np.zeros((nt,nci))
for i in range(nci):
    resu=ebm(epsilon,S0,alpha_type,ho,tsi[i],tpert=0,radp=0,ifig=False) # time integration of the model
    temp[:,i]=resu['T']
    
plt.figure(figsize=(10,6))
zerodk=273.15
plt.plot(resu['time'],temp-zerodk)
plt.xlabel('Time (years)',fontsize=14)
plt.ylabel('degrees Celcius',fontsize=14)
plt.title('Global average surface temperature',fontsize=14);

In [ ]:
# (b) fixed points and stability
ho=10.0;S0=1365;epsilon=0.61;alpha_type='nonlinear';
teq_stab,teq_unst=ebm_fixed_points(epsilon,S0,alpha_type,radp=0)
print ("stable fixed point =", '%.4f' %teq_stab[0] ," K")
if (len(teq_stab)==2): print ("stable fixed point =", '%.4f' %teq_stab[1] ," K")
if (len(teq_unst)==1): print ("unstable fixed point =", '%.4f' %teq_unst[0] ," K")

In [ ]:
#   stability of fixed points: we add a small amplitude perturbation around fixed points
#   and compute the climate feedbacks
Ti=teq_stab[1] # fixed point of the "present-day" climate
tpert=0.1 # initial temperature perturbation (could be either >0 or <0)
ebm(epsilon,S0,alpha_type,ho,Ti,tpert,radp=0,ifig=True);
feedback_factors(Ti, alpha_type, epsilon, S0);

In [ ]:
# (c) Add a large amplitude perturbation to the present day climate
#    what is the initial temperature anomaly required to switch from the present-day to the snowball state ?
#    Justify your answer.

In [ ]:
# (d) sensitivity to initial conditions for different emissivity values
#   the objective here is to determine the window of emissivity values for which multiple equilibria exist
epsilon=0.61;
tsi=np.linspace(230,300,nci)
nt=1000;temp=np.zeros((nt,nci))
for i in range(nci):
    resu=ebm(epsilon,S0,alpha_type,ho,tsi[i],0,radp=0,ifig=False) # time integration of the model
    temp[:,i]=resu['T']
    
plt.figure(figsize=(10,6))
zerodk=273.15
plt.plot(resu['time'],temp-zerodk)
plt.xlabel('Time (years)',fontsize=14)
plt.ylabel('degrees Celcius',fontsize=14)
plt.title('Global average surface temperature',fontsize=14);

In [ ]:
# ASR and OLR as a function of surface temperature
  
epsilon=0.61

zerodk=273.15
temp=np.linspace(180,350,1000)
plt.figure(figsize=(10,6))
asr,olr,qnet=qnet_toa(temp,epsilon,S0,alpha_type,radp=0)
plt.plot(temp-zerodk,asr,'r',label='ASR')
plt.plot(temp-zerodk,olr,'b',label='OLR with $\epsilon$='+str(epsilon))
ll=plt.legend(loc='upper left',fontsize=14)
plt.xlabel('Temperature (deg.C)',fontsize=14)
plt.ylabel('W/m2',fontsize=14)
plt.title('Sensitivity to emissivity $\epsilon$',fontsize=14);


In [ ]:
# (e) bifurcation structure. control parameter = emissivity
n=300
S0=1365.0
epsilon=np.linspace(0.3,0.8,n)
alpha_type='nonlinear' #albedo type
ebm_bifurcation_diagram(S0,epsilon,alpha_type,radp=0)

In [ ]:
# (f) Bifurcation diagram: control parameter = solar constant
n=300
S=S0*np.linspace(0.7,1.8,n)
alpha_type='nonlinear' #albedo type
epsilon=0.61
ebm_bifurcation_diagram(S,epsilon,alpha_type,radp=0)

In [ ]:
# (g) Response to a perturbation in the radiation balance at TOA

epsilon=0.61;S0=1365.0;alpha_type='nonlinear'
radp=np.linspace(0,10,11); # radiative forcing W/m2
teq_stab,teq_unst=ebm_fixed_points(epsilon,S0,alpha_type,radp=0)
Tsi=teq_stab[1]
ts=[]
for i in range(len(radp)):
    teq_stab,teq_unst=ebm_fixed_points(epsilon,S0,alpha_type,radp[i])
    ts.append(teq_stab[1])
resu=feedback_factors(Tsi,alpha_type,epsilon,S0) # negative feedback
delta_T = - np.array(radp)/resu['lambda_all']

plt.figure(figsize=(10,6))
plt.plot(radp,ts-Tsi,color='skyblue',marker='o',label='nonlinear model')
plt.plot(radp,delta_T,color='orange',marker='o',label=r'linear theory: $\Delta T_s = -R/\lambda$')
plt.legend(loc='best',fontsize=14)
plt.xlabel('Radiative forcing (W m$^{-2}$)',fontsize=14)
plt.ylabel('degrees Celcius',fontsize=14)
plt.title('Global average equilibrium surface temperature change',fontsize=14);